In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/

# 他社債務(12か月)

## 参考
https://www.kaggle.com/code/willkoehrsen/introduction-to-manual-feature-engineering

## モジュールとCSVの読み込み

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5604730604044291133
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1745276110
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11043864393000311506
 physical_device_desc: "device: 0, name: NVIDIA RTX A2000 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [4]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA RTX A2000 Laptop GPU


In [5]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import datawig
import mxnet as mx

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.datasets import make_moons

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

print("OK\n")

OK



In [6]:
app_train_origin = pd.read_csv('./home-credit-default-risk/exports/app_train.csv')
app_test_origin =  pd.read_csv('./home-credit-default-risk/exports/app_test.csv')
bureau_origin = pd.read_csv('./home-credit-default-risk/bureau.csv')
bureau_balance_origin = pd.read_csv('./home-credit-default-risk/bureau_balance.csv')
app_br_train_origin = pd.read_csv('./home-credit-default-risk/exports/app_train_br_all-domein_12-monthes.csv', na_values = 'na', dtype = 'object')
app_br_test_origin = pd.read_csv('./home-credit-default-risk/exports/app_test_br_all-domein_12-monthes.csv', na_values = 'na', dtype = 'object')
#POS_CASH_balance_origin = pd.read_csv('./home-credit-default-risk/POS_CASH_balance.csv')
#credit_card_balance_origin = pd.read_csv('./home-credit-default-risk/credit_card_balance.csv')
#previous_application_origin = pd.read_csv('./home-credit-default-risk/previous_application.csv')
#installments_payments_origin = pd.read_csv('./home-credit-default-risk/installments_payments.csv')
# custo data
#app_br_train_origin = pd.read_csv('./home-credit-default-risk/datasets/app_train_Loan_From_other_banks_cash-consumer-loans.csv')
#app_br_test_origin = pd.read_csv('./home-credit-default-risk/datasets/app_test_Loan_From_other_banks_cash-consumer-loans.csv')
#app_train_br_overdue_hist_origin = pd.read_csv('./home-credit-default-risk/datasets/app_train_br_overdue_hist.csv')
#app_test_br_overdue_hist_origin = pd.read_csv('./home-credit-default-risk/datasets/app_test_br_overdue_hist.csv')
#app_train_br_debt_credit_ratio_none_origin = pd.read_csv('./home-credit-default-risk/datasets/app_train_br_debt_credit_ratio_none.csv')
#app_test_br_debt_credit_ratio_none_origin = pd.read_csv('./home-credit-default-risk/datasets/app_test_br_debt_credit_ratio_none.csv')

app_br_test_origin.head()

,sk_id_curr,br_min_credit_day_overdue,br_max_credit_day_overdue,br_sum_credit_day_overdue,br_avg_credit_day_overdue,br_cnt_credit_day_overdue,br_stddev_credit_day_overdue,br_stddev_pop_credit_day_overdue,br_var_pop_credit_day_overdue,br_var_samp_credit_day_overdue,...,br_min_amt_annuity,br_max_amt_annuity,br_sum_amt_annuity,br_avg_amt_annuity,br_cnt_amt_annuity,br_stddev_amt_annuity,br_stddev_pop_amt_annuity,br_var_pop_amt_annuity,br_var_samp_amt_annuity,br_variance_pop_amt_annuity
0,100001,0,0,0,0.0000,172,0,0,0,0,...,0.0,9364.5,212634,1236.2441860465117,172,2794.7282221572123,2794.7282221572123,7810505.835722012,7856181.308445533,7810505.835722012
1,100005,0,0,0,0.0000,21,0,0,0,0,...,0.0,4261.5,12784.5,608.7857142857143,21,1491.2143626957877,1491.2143626957877,2223720.2755102045,2334906.2892857147,2223720.2755102045
2,100013,0,0,0,0.0000,217,0,0,0,0,...,NaN,0.0,0,0,217,0,0,0,0,0
3,100028,0,0,0,0.0000,536,0,0,0,0,...,NaN,6562.485,443230.7849999995,826.9231063432827,536,2284.0107511879555,2284.0107511879555,5216705.111542168,5226455.9622179475,5216705.111542168
4,100042,0,0,0,0.0000,547,0,0,0,0,...,NaN,40500.0,4324419,7905.702010968921,547,15016.22619291047,15016.22619291047,225487049.07665047,225900029.02001432,225487049.07665047


In [7]:
app_train = app_train_origin.copy()
app_test = app_test_origin.copy()
bureau = bureau_origin.copy()
bureau_balance = bureau_balance_origin.copy()
app_br_train = app_br_train_origin.copy()
app_br_test = app_br_test_origin.copy()
app_br_test.head()

,sk_id_curr,br_min_credit_day_overdue,br_max_credit_day_overdue,br_sum_credit_day_overdue,br_avg_credit_day_overdue,br_cnt_credit_day_overdue,br_stddev_credit_day_overdue,br_stddev_pop_credit_day_overdue,br_var_pop_credit_day_overdue,br_var_samp_credit_day_overdue,...,br_min_amt_annuity,br_max_amt_annuity,br_sum_amt_annuity,br_avg_amt_annuity,br_cnt_amt_annuity,br_stddev_amt_annuity,br_stddev_pop_amt_annuity,br_var_pop_amt_annuity,br_var_samp_amt_annuity,br_variance_pop_amt_annuity
0,100001,0,0,0,0.0000,172,0,0,0,0,...,0.0,9364.5,212634,1236.2441860465117,172,2794.7282221572123,2794.7282221572123,7810505.835722012,7856181.308445533,7810505.835722012
1,100005,0,0,0,0.0000,21,0,0,0,0,...,0.0,4261.5,12784.5,608.7857142857143,21,1491.2143626957877,1491.2143626957877,2223720.2755102045,2334906.2892857147,2223720.2755102045
2,100013,0,0,0,0.0000,217,0,0,0,0,...,NaN,0.0,0,0,217,0,0,0,0,0
3,100028,0,0,0,0.0000,536,0,0,0,0,...,NaN,6562.485,443230.7849999995,826.9231063432827,536,2284.0107511879555,2284.0107511879555,5216705.111542168,5226455.9622179475,5216705.111542168
4,100042,0,0,0,0.0000,547,0,0,0,0,...,NaN,40500.0,4324419,7905.702010968921,547,15016.22619291047,15016.22619291047,225487049.07665047,225900029.02001432,225487049.07665047


In [8]:
restore_columns = list(app_br_test.columns)
restore_columns.remove('sk_id_curr')
restore_columns

['br_min_credit_day_overdue',
 'br_max_credit_day_overdue',
 'br_sum_credit_day_overdue',
 'br_avg_credit_day_overdue',
 'br_cnt_credit_day_overdue',
 'br_stddev_credit_day_overdue',
 'br_stddev_pop_credit_day_overdue',
 'br_var_pop_credit_day_overdue',
 'br_var_samp_credit_day_overdue',
 'br_variance_pop_credit_day_overdue',
 'br_min_days_credit_enddate',
 'br_max_days_credit_enddate',
 'br_sum_days_credit_enddate',
 'br_avg_days_credit_enddate',
 'br_cnt_days_credit_enddate',
 'br_stddev_days_credit_enddate',
 'br_stddev_pop_days_credit_enddate',
 'br_var_pop_days_credit_enddate',
 'br_var_samp_days_credit_enddate',
 'br_variance_pop_days_credit_enddate',
 'br_min_days_enddate_fact',
 'br_max_days_enddate_fact',
 'br_sum_days_enddate_fact',
 'br_avg_days_enddate_fact',
 'br_cnt_days_enddate_fact',
 'br_stddev_days_enddate_fact',
 'br_stddev_pop_days_enddate_fact',
 'br_var_pop_days_enddate_fact',
 'br_var_samp_days_enddate_fact',
 'br_variance_pop_days_enddate_fact',
 'br_min_amt_cre

In [9]:
# brの数値データをfloatへキャスト
app_br_train[restore_columns] = app_br_train[restore_columns].astype('float64')
app_br_test[restore_columns] = app_br_test[restore_columns].astype('float64')
app_br_test.head()

,sk_id_curr,br_min_credit_day_overdue,br_max_credit_day_overdue,br_sum_credit_day_overdue,br_avg_credit_day_overdue,br_cnt_credit_day_overdue,br_stddev_credit_day_overdue,br_stddev_pop_credit_day_overdue,br_var_pop_credit_day_overdue,br_var_samp_credit_day_overdue,...,br_min_amt_annuity,br_max_amt_annuity,br_sum_amt_annuity,br_avg_amt_annuity,br_cnt_amt_annuity,br_stddev_amt_annuity,br_stddev_pop_amt_annuity,br_var_pop_amt_annuity,br_var_samp_amt_annuity,br_variance_pop_amt_annuity
0,100001,0.0,0.0,0.0,0.0,172.0,0.0,0.0,0.0,0.0,...,0.0,9364.500,212634.000,1236.244186,172.0,2794.728222,2794.728222,7.810506e+06,7.856181e+06,7.810506e+06
1,100005,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,...,0.0,4261.500,12784.500,608.785714,21.0,1491.214363,1491.214363,2.223720e+06,2.334906e+06,2.223720e+06
2,100013,0.0,0.0,0.0,0.0,217.0,0.0,0.0,0.0,0.0,...,NaN,0.000,0.000,0.000000,217.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
3,100028,0.0,0.0,0.0,0.0,536.0,0.0,0.0,0.0,0.0,...,NaN,6562.485,443230.785,826.923106,536.0,2284.010751,2284.010751,5.216705e+06,5.226456e+06,5.216705e+06
4,100042,0.0,0.0,0.0,0.0,547.0,0.0,0.0,0.0,0.0,...,NaN,40500.000,4324419.000,7905.702011,547.0,15016.226193,15016.226193,2.254870e+08,2.259000e+08,2.254870e+08


In [10]:
# infを未定義値へ置換
app_br_train = app_br_train.replace([np.inf, -np.inf], np.nan)
app_br_test = app_br_test.replace([np.inf, -np.inf], np.nan)
app_br_test.head()

,sk_id_curr,br_min_credit_day_overdue,br_max_credit_day_overdue,br_sum_credit_day_overdue,br_avg_credit_day_overdue,br_cnt_credit_day_overdue,br_stddev_credit_day_overdue,br_stddev_pop_credit_day_overdue,br_var_pop_credit_day_overdue,br_var_samp_credit_day_overdue,...,br_min_amt_annuity,br_max_amt_annuity,br_sum_amt_annuity,br_avg_amt_annuity,br_cnt_amt_annuity,br_stddev_amt_annuity,br_stddev_pop_amt_annuity,br_var_pop_amt_annuity,br_var_samp_amt_annuity,br_variance_pop_amt_annuity
0,100001,0.0,0.0,0.0,0.0,172.0,0.0,0.0,0.0,0.0,...,0.0,9364.500,212634.000,1236.244186,172.0,2794.728222,2794.728222,7.810506e+06,7.856181e+06,7.810506e+06
1,100005,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,...,0.0,4261.500,12784.500,608.785714,21.0,1491.214363,1491.214363,2.223720e+06,2.334906e+06,2.223720e+06
2,100013,0.0,0.0,0.0,0.0,217.0,0.0,0.0,0.0,0.0,...,NaN,0.000,0.000,0.000000,217.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
3,100028,0.0,0.0,0.0,0.0,536.0,0.0,0.0,0.0,0.0,...,NaN,6562.485,443230.785,826.923106,536.0,2284.010751,2284.010751,5.216705e+06,5.226456e+06,5.216705e+06
4,100042,0.0,0.0,0.0,0.0,547.0,0.0,0.0,0.0,0.0,...,NaN,40500.000,4324419.000,7905.702011,547.0,15016.226193,15016.226193,2.254870e+08,2.259000e+08,2.254870e+08


In [11]:
# sk_id_currをINT型へキャスト
app_br_train['sk_id_curr'] = app_br_train['sk_id_curr'].astype(int)
app_br_test['sk_id_curr'] = app_br_test['sk_id_curr'].astype(int)
app_br_test.head()

,sk_id_curr,br_min_credit_day_overdue,br_max_credit_day_overdue,br_sum_credit_day_overdue,br_avg_credit_day_overdue,br_cnt_credit_day_overdue,br_stddev_credit_day_overdue,br_stddev_pop_credit_day_overdue,br_var_pop_credit_day_overdue,br_var_samp_credit_day_overdue,...,br_min_amt_annuity,br_max_amt_annuity,br_sum_amt_annuity,br_avg_amt_annuity,br_cnt_amt_annuity,br_stddev_amt_annuity,br_stddev_pop_amt_annuity,br_var_pop_amt_annuity,br_var_samp_amt_annuity,br_variance_pop_amt_annuity
0,100001,0.0,0.0,0.0,0.0,172.0,0.0,0.0,0.0,0.0,...,0.0,9364.500,212634.000,1236.244186,172.0,2794.728222,2794.728222,7.810506e+06,7.856181e+06,7.810506e+06
1,100005,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,...,0.0,4261.500,12784.500,608.785714,21.0,1491.214363,1491.214363,2.223720e+06,2.334906e+06,2.223720e+06
2,100013,0.0,0.0,0.0,0.0,217.0,0.0,0.0,0.0,0.0,...,NaN,0.000,0.000,0.000000,217.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
3,100028,0.0,0.0,0.0,0.0,536.0,0.0,0.0,0.0,0.0,...,NaN,6562.485,443230.785,826.923106,536.0,2284.010751,2284.010751,5.216705e+06,5.226456e+06,5.216705e+06
4,100042,0.0,0.0,0.0,0.0,547.0,0.0,0.0,0.0,0.0,...,NaN,40500.000,4324419.000,7905.702011,547.0,15016.226193,15016.226193,2.254870e+08,2.259000e+08,2.254870e+08


In [12]:
# Plots the disribution of a variable colored by value of the target
def kde_target(var_name, df):
    
    # Calculate the correlation coefficient between the new variable and the target
    corr = df['TARGET'].corr(df[var_name])
    
    # Calculate medians for repaid vs not repaid
    avg_repaid = df.loc[df['TARGET'] == 0, var_name].median()
    avg_not_repaid = df.loc[df['TARGET'] == 1, var_name].median()
    
    plt.figure(figsize = (12, 6))
    
    # Plot the distribution for target == 0 and target == 1
    sns.kdeplot(df.loc[df['TARGET'] == 0, var_name], label = 'TARGET == 0')
    sns.kdeplot(df.loc[df['TARGET'] == 1, var_name], label = 'TARGET == 1')
    
    # label the plot
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend();
    
    # print out the correlation
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    # Print out average values
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)

## データ読み込み

In [13]:
len(app_train)

307511

In [14]:
len(app_test)

48744

In [15]:
len(app_br_train)

91932

In [16]:
len(app_br_test)

42311

In [17]:
len(app_br_test)

42311

In [18]:
# applicationの欠損値を補完
imputer = IterativeImputer()
replace_columns = list(app_test.select_dtypes(include='number').columns)
app_train = pd.DataFrame(imputer.fit_transform(app_train[replace_columns].copy()), columns=replace_columns)
app_test = pd.DataFrame(imputer.fit_transform(app_test[replace_columns].copy()), columns=replace_columns)
app_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,...,FLAG_DOCUMENT_13,FLAG_DOCUMENT_16,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_YEAR,CREDIT_TERM,DAYS_EMPLOYED_PERCENT,credit_annuity_ratio,credit_goods_price_ratio,credit_downpayment,AGE_INT
0,100001.0,0.0,0.0,0.0,0.0,135000.0,568800.0,20560.5,450000.0,0.0,...,0.0,0.0,0.000000,0.000000,0.036147,0.121044,19.775662,1.2640,-118800.0,52.715068
1,100005.0,0.0,1.0,0.0,0.0,99000.0,222768.0,17370.0,180000.0,0.0,...,0.0,0.0,0.000000,3.000000,0.077973,0.247398,74.467617,1.2376,-42768.0,49.490411
2,100013.0,0.0,1.0,1.0,0.0,202500.0,663264.0,69777.0,630000.0,0.0,...,0.0,0.0,0.000000,4.000000,0.105202,0.222477,1.934735,1.0528,-33264.0,54.898630
3,100028.0,0.0,0.0,0.0,2.0,315000.0,1575000.0,49018.5,1575000.0,0.0,...,0.0,0.0,0.000000,3.000000,0.031123,0.133515,6.378867,1.0000,0.0,38.290411
4,100038.0,0.0,1.0,1.0,1.0,180000.0,625500.0,32067.0,625500.0,0.0,...,0.0,0.0,0.010242,1.748333,0.051266,0.168021,15.997755,1.0000,0.0,35.726027


In [19]:
app_train['SK_ID_CURR'] = app_train[["SK_ID_CURR"]].astype(int) 
app_test['SK_ID_CURR'] = app_test[["SK_ID_CURR"]].astype(int)
app_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,...,FLAG_DOCUMENT_13,FLAG_DOCUMENT_16,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_YEAR,CREDIT_TERM,DAYS_EMPLOYED_PERCENT,credit_annuity_ratio,credit_goods_price_ratio,credit_downpayment,AGE_INT
0,100001,0.0,0.0,0.0,0.0,135000.0,568800.0,20560.5,450000.0,0.0,...,0.0,0.0,0.000000,0.000000,0.036147,0.121044,19.775662,1.2640,-118800.0,52.715068
1,100005,0.0,1.0,0.0,0.0,99000.0,222768.0,17370.0,180000.0,0.0,...,0.0,0.0,0.000000,3.000000,0.077973,0.247398,74.467617,1.2376,-42768.0,49.490411
2,100013,0.0,1.0,1.0,0.0,202500.0,663264.0,69777.0,630000.0,0.0,...,0.0,0.0,0.000000,4.000000,0.105202,0.222477,1.934735,1.0528,-33264.0,54.898630
3,100028,0.0,0.0,0.0,2.0,315000.0,1575000.0,49018.5,1575000.0,0.0,...,0.0,0.0,0.000000,3.000000,0.031123,0.133515,6.378867,1.0000,0.0,38.290411
4,100038,0.0,1.0,1.0,1.0,180000.0,625500.0,32067.0,625500.0,0.0,...,0.0,0.0,0.010242,1.748333,0.051266,0.168021,15.997755,1.0000,0.0,35.726027


In [20]:
app_train['EXT_SOURCE_1'].isnull().sum()

0

In [21]:
#app_train = app_train[['SK_ID_CURR','TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']].copy()
#app_test = app_test[['SK_ID_CURR','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']].copy()
#app_train = app_train[['SK_ID_CURR','TARGET']].copy()
#app_test = app_test[['SK_ID_CURR']].copy()
#len(app_test)

In [22]:
app_br_train = app_br_train.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_br_test = app_br_test.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
len(app_br_test)

42311

In [23]:
app_br_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42311 entries, 0 to 42310
Columns: 111 entries, SK_ID_CURR to br_variance_pop_amt_annuity
dtypes: float64(110), int32(1)
memory usage: 35.7 MB


In [24]:
# trainへマージ
app_train = pd.merge(app_train, app_br_train, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, app_br_test, on='SK_ID_CURR', how='left')
len(app_test)

48744

In [25]:
app_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,...,br_min_amt_annuity,br_max_amt_annuity,br_sum_amt_annuity,br_avg_amt_annuity,br_cnt_amt_annuity,br_stddev_amt_annuity,br_stddev_pop_amt_annuity,br_var_pop_amt_annuity,br_var_samp_amt_annuity,br_variance_pop_amt_annuity
0,100001,0.0,0.0,0.0,0.0,135000.0,568800.0,20560.5,450000.0,0.0,...,0.0,9364.500,212634.000,1236.244186,172.0,2794.728222,2794.728222,7.810506e+06,7.856181e+06,7.810506e+06
1,100005,0.0,1.0,0.0,0.0,99000.0,222768.0,17370.0,180000.0,0.0,...,0.0,4261.500,12784.500,608.785714,21.0,1491.214363,1491.214363,2.223720e+06,2.334906e+06,2.223720e+06
2,100013,0.0,1.0,1.0,0.0,202500.0,663264.0,69777.0,630000.0,0.0,...,NaN,0.000,0.000,0.000000,217.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
3,100028,0.0,0.0,0.0,2.0,315000.0,1575000.0,49018.5,1575000.0,0.0,...,NaN,6562.485,443230.785,826.923106,536.0,2284.010751,2284.010751,5.216705e+06,5.226456e+06,5.216705e+06
4,100038,0.0,1.0,1.0,1.0,180000.0,625500.0,32067.0,625500.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 他金融機関での借り入れ情報(bureau)前処理

### 欠損値の補完

In [26]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [27]:
app_br_test.select_dtypes(include='number')

,SK_ID_CURR,br_min_credit_day_overdue,br_max_credit_day_overdue,br_sum_credit_day_overdue,br_avg_credit_day_overdue,br_cnt_credit_day_overdue,br_stddev_credit_day_overdue,br_stddev_pop_credit_day_overdue,br_var_pop_credit_day_overdue,br_var_samp_credit_day_overdue,...,br_min_amt_annuity,br_max_amt_annuity,br_sum_amt_annuity,br_avg_amt_annuity,br_cnt_amt_annuity,br_stddev_amt_annuity,br_stddev_pop_amt_annuity,br_var_pop_amt_annuity,br_var_samp_amt_annuity,br_variance_pop_amt_annuity
0,100001,0.0,0.0,0.0,0.0,172.0,0.0,0.0,0.0,0.0,...,0.000,9364.500,2.126340e+05,1236.244186,172.0,2794.728222,2794.728222,7.810506e+06,7.856181e+06,7.810506e+06
1,100005,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,...,0.000,4261.500,1.278450e+04,608.785714,21.0,1491.214363,1491.214363,2.223720e+06,2.334906e+06,2.223720e+06
2,100013,0.0,0.0,0.0,0.0,217.0,0.0,0.0,0.0,0.0,...,NaN,0.000,0.000000e+00,0.000000,217.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
3,100028,0.0,0.0,0.0,0.0,536.0,0.0,0.0,0.0,0.0,...,NaN,6562.485,4.432308e+05,826.923106,536.0,2284.010751,2284.010751,5.216705e+06,5.226456e+06,5.216705e+06
4,100042,0.0,0.0,0.0,0.0,547.0,0.0,0.0,0.0,0.0,...,NaN,40500.000,4.324419e+06,7905.702011,547.0,15016.226193,15016.226193,2.254870e+08,2.259000e+08,2.254870e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42306,456202,0.0,0.0,0.0,0.0,85.0,0.0,0.0,0.0,0.0,...,NaN,0.000,0.000000e+00,0.000000,85.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
42307,456221,0.0,0.0,0.0,0.0,121.0,0.0,0.0,0.0,0.0,...,0.000,17257.500,4.706100e+05,3889.338843,121.0,6863.537346,6863.537346,4.710814e+07,4.750071e+07,4.710814e+07
42308,456223,0.0,0.0,0.0,0.0,92.0,0.0,0.0,0.0,0.0,...,NaN,5053.500,4.042800e+04,439.434783,92.0,1423.931440,1423.931440,2.027581e+06,2.049862e+06,2.027581e+06
42309,456224,0.0,0.0,0.0,0.0,800.0,0.0,0.0,0.0,0.0,...,NaN,4900.500,1.831856e+06,2289.819375,800.0,6466.382400,6466.382400,4.181410e+07,4.186643e+07,4.181410e+07


In [28]:
input_columns = list(app_br_test.select_dtypes(include='number').columns)
input_columns.remove('SK_ID_CURR')
input_columns

['br_min_credit_day_overdue',
 'br_max_credit_day_overdue',
 'br_sum_credit_day_overdue',
 'br_avg_credit_day_overdue',
 'br_cnt_credit_day_overdue',
 'br_stddev_credit_day_overdue',
 'br_stddev_pop_credit_day_overdue',
 'br_var_pop_credit_day_overdue',
 'br_var_samp_credit_day_overdue',
 'br_variance_pop_credit_day_overdue',
 'br_min_days_credit_enddate',
 'br_max_days_credit_enddate',
 'br_sum_days_credit_enddate',
 'br_avg_days_credit_enddate',
 'br_cnt_days_credit_enddate',
 'br_stddev_days_credit_enddate',
 'br_stddev_pop_days_credit_enddate',
 'br_var_pop_days_credit_enddate',
 'br_var_samp_days_credit_enddate',
 'br_variance_pop_days_credit_enddate',
 'br_min_days_enddate_fact',
 'br_max_days_enddate_fact',
 'br_sum_days_enddate_fact',
 'br_avg_days_enddate_fact',
 'br_cnt_days_enddate_fact',
 'br_stddev_days_enddate_fact',
 'br_stddev_pop_days_enddate_fact',
 'br_var_pop_days_enddate_fact',
 'br_var_samp_days_enddate_fact',
 'br_variance_pop_days_enddate_fact',
 'br_min_amt_cre

In [29]:
output_columns = list(app_test.select_dtypes(include='number').columns)
output_columns.remove('SK_ID_CURR')
output_columns

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'OCCUPATION_TYPE',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_M

In [30]:
import datawig

imputer = datawig.SimpleImputer(
    input_columns=input_columns, # List: どのカラムで代入モデルを作成するか
    output_column=output_columns[0], # どのカラムの欠損補完を行うか
    output_path='imputer_model', # 代入モデルの出力先
    num_hash_buckets=int(2 ** 15),
    num_labels=100,
    tokens='chars',
    numeric_latent_dim=100,
    numeric_hidden_layers=1,
    is_explainable=False
)

In [31]:
app_train = datawig.SimpleImputer.complete(app_train,
                                    precision_threshold=0.0,
                                    inplace=False,
                                    hpo=False,
                                    verbose=0,
                                    num_epochs=100,
                                    iterations=1,
                                    output_path='.'
                                   )

KeyboardInterrupt: 

In [ ]:
app_test = datawig.SimpleImputer.complete(app_test,
                                    precision_threshold=0.0,
                                    inplace=False,
                                    hpo=False,
                                    verbose=0,
                                    num_epochs=100,
                                    iterations=1,
                                    output_path='.'
                                   )

In [ ]:
remove_columns = list(app_test_origin.columns)
remove_columns.remove('SK_ID_CURR')
remove_columns.remove('DAYS_EMPLOYED_ANOM')

In [ ]:
app_train.drop(remove_columns, axis=1, inplace=True)
app_test.drop(remove_columns, axis=1, inplace=True)
app_test.head()

In [ ]:
### train
app_train.to_csv(
    path_or_buf="./home-credit-default-risk/exports/hc_br_train_priprocess.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [ ]:
### test
app_test.to_csv(
    path_or_buf="./home-credit-default-risk/exports/app_test_priprocess.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [ ]:
app_train[replace_columns] = app_train_replace[replace_columns]
app_test[replace_columns] = app_test_replace[replace_columns]

In [ ]:
app_train_replace.head()

In [ ]:
app_test_replace.head()

In [ ]:
app_train[replace_columns] = app_train_replace[replace_columns]
app_test[replace_columns] = app_test_replace[replace_columns]
app_test.head()

In [ ]:
remove_columns = list(app_test_origin.columns)
remove_columns.remove('SK_ID_CURR')
remove_columns

In [ ]:
app_train.drop(remove_columns, axis=1, inplace=True)
app_test.drop(remove_columns, axis=1, inplace=True)
app_train.head()

### br_min_credit_day_overdue

In [ ]:
app_train[['TARGET', 'br_min_credit_day_overdue']].corr()

In [ ]:
kde_target('br_min_credit_day_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_min_credit_day_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_min_credit_day_overdue'] = pt.transform(app_train[num_cols])
app_test['br_min_credit_day_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## 標準化
num_cols = ['br_min_credit_day_overdue']
scaler = StandardScaler()
scaler.fit(app_train[num_cols])

app_train[num_cols] = scaler.transform(app_train[num_cols])
app_test[num_cols] = scaler.transform(app_test[num_cols])
app_train[['br_min_credit_day_overdue', 'TARGET']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_min_credit_day_overdue', app_train)

### br_max_credit_day_overdue

In [ ]:
app_train[['TARGET', 'br_max_credit_day_overdue']].corr()

In [ ]:
kde_target('br_max_credit_day_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_max_credit_day_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_max_credit_day_overdue'] = pt.transform(app_train[num_cols])
app_test['br_max_credit_day_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_max_credit_day_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_max_credit_day_overdue', app_train)

### br_sum_credit_day_overdue

In [ ]:
app_train[['TARGET', 'br_sum_credit_day_overdue']].corr()

In [ ]:
kde_target('br_sum_credit_day_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_sum_credit_day_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_sum_credit_day_overdue'] = pt.transform(app_train[num_cols])
app_test['br_sum_credit_day_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_sum_credit_day_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_sum_credit_day_overdue', app_train)

### br_avg_credit_day_overdue

In [ ]:
app_train[['TARGET', 'br_avg_credit_day_overdue']].corr()

In [ ]:
kde_target('br_avg_credit_day_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_avg_credit_day_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_avg_credit_day_overdue'] = pt.transform(app_train[num_cols])
app_test['br_avg_credit_day_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_avg_credit_day_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_avg_credit_day_overdue', app_train)

### br_cnt_credit_day_overdue

In [ ]:
app_train[['TARGET', 'br_cnt_credit_day_overdue']].corr()

In [ ]:
kde_target('br_cnt_credit_day_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_cnt_credit_day_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_cnt_credit_day_overdue'] = pt.transform(app_train[num_cols])
app_test['br_cnt_credit_day_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_cnt_credit_day_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_cnt_credit_day_overdue', app_train)

### br_stddev_credit_day_overdue

In [ ]:
app_train[['TARGET', 'br_stddev_credit_day_overdue']].corr()

In [ ]:
kde_target('br_stddev_credit_day_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_credit_day_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_credit_day_overdue'] = pt.transform(app_train[num_cols])
app_test['br_stddev_credit_day_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_credit_day_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_credit_day_overdue', app_train)

### br_stddev_pop_credit_day_overdue

In [ ]:
app_train[['TARGET', 'br_stddev_pop_credit_day_overdue']].corr()

In [ ]:
kde_target('br_stddev_pop_credit_day_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_pop_credit_day_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_pop_credit_day_overdue'] = pt.transform(app_train[num_cols])
app_test['br_stddev_pop_credit_day_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_pop_credit_day_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_pop_credit_day_overdue', app_train)

### br_var_pop_credit_day_overdue

In [ ]:
app_train[['TARGET', 'br_var_pop_credit_day_overdue']].corr()

In [ ]:
kde_target('br_var_pop_credit_day_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_pop_credit_day_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_pop_credit_day_overdue'] = pt.transform(app_train[num_cols])
app_test['br_var_pop_credit_day_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_pop_credit_day_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_pop_credit_day_overdue', app_train)

### br_var_samp_credit_day_overdue

In [ ]:
app_train[['TARGET', 'br_var_samp_credit_day_overdue']].corr()

In [ ]:
kde_target('br_var_samp_credit_day_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_samp_credit_day_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_samp_credit_day_overdue'] = pt.transform(app_train[num_cols])
app_test['br_var_samp_credit_day_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_samp_credit_day_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_samp_credit_day_overdue', app_train)

### br_variance_pop_credit_day_overdue

In [ ]:
app_train[['TARGET', 'br_variance_pop_credit_day_overdue']].corr()

In [ ]:
kde_target('br_variance_pop_credit_day_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_variance_pop_credit_day_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_variance_pop_credit_day_overdue'] = pt.transform(app_train[num_cols])
app_test['br_variance_pop_credit_day_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_variance_pop_credit_day_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_variance_pop_credit_day_overdue', app_train)

### br_min_days_credit_enddate

In [ ]:
app_train[['TARGET', 'br_min_days_credit_enddate']].corr()

In [ ]:
kde_target('br_min_days_credit_enddate', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_min_days_credit_enddate']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_min_days_credit_enddate'] = pt.transform(app_train[num_cols])
app_test['br_min_days_credit_enddate'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_min_days_credit_enddate']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_min_days_credit_enddate', app_train)

### br_max_days_credit_enddate

In [ ]:
app_train[['TARGET', 'br_max_days_credit_enddate']].corr()

In [ ]:
kde_target('br_max_days_credit_enddate', app_train)

In [ ]:
## Yao-Johnson変換
#num_cols = ['br_max_days_credit_enddate']
### 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
#app_train['br_max_days_credit_enddate'] = pt.transform(app_train[num_cols])
#app_test['br_max_days_credit_enddate'] = pt.transform(app_test[num_cols])


In [ ]:
## Yao-Johnson変換後の目的変数との相関
#app_train[['TARGET', 'br_max_days_credit_enddate']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
#kde_target('br_max_days_credit_enddate', app_train)

### br_sum_days_credit_enddate

In [ ]:
app_train[['TARGET', 'br_sum_days_credit_enddate']].corr()

In [ ]:
kde_target('br_sum_days_credit_enddate', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_sum_days_credit_enddate']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_sum_days_credit_enddate'] = pt.transform(app_train[num_cols])
app_test['br_sum_days_credit_enddate'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_sum_days_credit_enddate']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_sum_days_credit_enddate', app_train)

### br_avg_days_credit_enddate

In [ ]:
app_train[['TARGET', 'br_avg_days_credit_enddate']].corr()

In [ ]:
kde_target('br_avg_days_credit_enddate', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_avg_days_credit_enddate']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_avg_days_credit_enddate'] = pt.transform(app_train[num_cols])
app_test['br_avg_days_credit_enddate'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_avg_days_credit_enddate']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_avg_days_credit_enddate', app_train)

### br_cnt_days_credit_enddate

In [ ]:
app_train[['TARGET', 'br_cnt_days_credit_enddate']].corr()

In [ ]:
kde_target('br_cnt_days_credit_enddate', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_cnt_days_credit_enddate']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_cnt_days_credit_enddate'] = pt.transform(app_train[num_cols])
app_test['br_cnt_days_credit_enddate'] = pt.transform(app_test[num_cols])


In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_cnt_days_credit_enddate']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_cnt_days_credit_enddate', app_train)

### br_stddev_days_credit_enddate

In [ ]:
app_train[['TARGET', 'br_stddev_days_credit_enddate']].corr()

In [ ]:
kde_target('br_stddev_days_credit_enddate', app_train)

In [ ]:
## Yao-Johnson変換
#num_cols = ['br_stddev_days_credit_enddate']
### 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
#app_train['br_stddev_days_credit_enddate'] = pt.transform(app_train[num_cols])
#app_test['br_stddev_days_credit_enddate'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
#app_train[['TARGET', 'br_stddev_days_credit_enddate']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
#kde_target('br_stddev_days_credit_enddate', app_train)

### br_stddev_pop_days_credit_enddate

In [ ]:
app_train[['TARGET', 'br_stddev_pop_days_credit_enddate']].corr()

In [ ]:
kde_target('br_stddev_pop_days_credit_enddate', app_train)

In [ ]:
## Yao-Johnson変換
#num_cols = ['br_stddev_pop_days_credit_enddate']
### 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
#app_train['br_stddev_pop_days_credit_enddate'] = pt.transform(app_train[num_cols])
#app_test['br_stddev_pop_days_credit_enddate'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
#app_train[['TARGET', 'br_stddev_pop_days_credit_enddate']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
#kde_target('br_stddev_pop_days_credit_enddate', app_train)

### br_var_pop_days_credit_enddate

In [ ]:
app_train[['TARGET', 'br_var_pop_days_credit_enddate']].corr()

In [ ]:
kde_target('br_var_pop_days_credit_enddate', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_pop_days_credit_enddate']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_pop_days_credit_enddate'] = pt.transform(app_train[num_cols])
app_test['br_var_pop_days_credit_enddate'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_pop_days_credit_enddate']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_pop_days_credit_enddate', app_train)

### br_var_samp_days_credit_enddate

In [ ]:
app_train[['TARGET', 'br_var_samp_days_credit_enddate']].corr()

In [ ]:
kde_target('br_var_samp_days_credit_enddate', app_train)

In [ ]:
## Yao-Johnson変換
#num_cols = ['br_var_samp_days_credit_enddate']
### 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
#app_train['br_var_samp_days_credit_enddate'] = pt.transform(app_train[num_cols])
#app_test['br_var_samp_days_credit_enddate'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
#app_train[['TARGET', 'br_var_samp_days_credit_enddate']].corr()

In [ ]:

## Yao-Johnson変換後の特徴量の分布
#kde_target('br_var_samp_days_credit_enddate', app_train)

### br_variance_pop_days_credit_enddate

In [ ]:
app_train[['TARGET', 'br_variance_pop_days_credit_enddate']].corr()

In [ ]:
kde_target('br_variance_pop_days_credit_enddate', app_train)

### br_min_days_enddate_fact

In [ ]:
app_train[['TARGET', 'br_min_days_enddate_fact']].corr()

In [ ]:
kde_target('br_min_days_enddate_fact', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_min_days_enddate_fact']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_min_days_enddate_fact'] = pt.transform(app_train[num_cols])
app_test['br_min_days_enddate_fact'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_min_days_enddate_fact']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_min_days_enddate_fact', app_train)

### br_max_days_enddate_fact

In [ ]:
app_train[['TARGET', 'br_max_days_enddate_fact']].corr()

In [ ]:
kde_target('br_max_days_enddate_fact', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_max_days_enddate_fact']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_max_days_enddate_fact'] = pt.transform(app_train[num_cols])
app_test['br_max_days_enddate_fact'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_max_days_enddate_fact']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_max_days_enddate_fact', app_train)

### br_sum_days_enddate_fact

In [ ]:
app_train[['TARGET', 'br_sum_days_enddate_fact']].corr()

In [ ]:
kde_target('br_sum_days_enddate_fact', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_sum_days_enddate_fact']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_sum_days_enddate_fact'] = pt.transform(app_train[num_cols])
app_test['br_sum_days_enddate_fact'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_sum_days_enddate_fact']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_sum_days_enddate_fact', app_train)

### br_avg_days_enddate_fact

In [ ]:
app_train[['TARGET', 'br_avg_days_enddate_fact']].corr()

In [ ]:
kde_target('br_avg_days_enddate_fact', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_avg_days_enddate_fact']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_avg_days_enddate_fact'] = pt.transform(app_train[num_cols])
app_test['br_avg_days_enddate_fact'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_avg_days_enddate_fact']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_avg_days_enddate_fact', app_train)

### br_cnt_days_enddate_fact

In [ ]:
app_train[['TARGET', 'br_cnt_days_enddate_fact']].corr()

In [ ]:
kde_target('br_cnt_days_enddate_fact', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_cnt_days_enddate_fact']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_cnt_days_enddate_fact'] = pt.transform(app_train[num_cols])
app_test['br_cnt_days_enddate_fact'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_cnt_days_enddate_fact']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_cnt_days_enddate_fact', app_train)

### br_stddev_days_enddate_fact

In [ ]:
app_train[['TARGET', 'br_stddev_days_enddate_fact']].corr()

In [ ]:
kde_target('br_stddev_days_enddate_fact', app_train)

In [ ]:
## Yao-Johnson変換
#num_cols = ['br_stddev_days_enddate_fact']
### 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
#app_train['br_stddev_days_enddate_fact'] = pt.transform(app_train[num_cols])
#app_test['br_stddev_days_enddate_fact'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
#app_train[['TARGET', 'br_stddev_days_enddate_fact']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
#kde_target('br_stddev_days_enddate_fact', app_train)

### br_stddev_pop_days_enddate_fact

In [ ]:
app_train[['TARGET', 'br_stddev_pop_days_enddate_fact']].corr()

In [ ]:
kde_target('br_stddev_pop_days_enddate_fact', app_train)

In [ ]:
## Yao-Johnson変換
#num_cols = ['br_stddev_pop_days_enddate_fact']
### 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
#app_train['br_stddev_pop_days_enddate_fact'] = pt.transform(app_train[num_cols])
#app_test['br_stddev_pop_days_enddate_fact'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
#app_train[['TARGET', 'br_stddev_pop_days_enddate_fact']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_pop_days_enddate_fact', app_train)

### br_var_pop_days_enddate_fact

In [ ]:
app_train[['TARGET', 'br_var_pop_days_enddate_fact']].corr()

In [ ]:
kde_target('br_var_pop_days_enddate_fact', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_pop_days_enddate_fact']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_pop_days_enddate_fact'] = pt.transform(app_train[num_cols])
app_test['br_var_pop_days_enddate_fact'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_pop_days_enddate_fact']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_pop_days_enddate_fact', app_train)

### br_var_samp_days_enddate_fact

In [ ]:
app_train[['TARGET', 'br_var_samp_days_enddate_fact']].corr()

In [ ]:
kde_target('br_var_samp_days_enddate_fact', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_samp_days_enddate_fact']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_samp_days_enddate_fact'] = pt.transform(app_train[num_cols])
app_test['br_var_samp_days_enddate_fact'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_samp_days_enddate_fact']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_samp_days_enddate_fact', app_train)

### br_variance_pop_days_enddate_fact

In [ ]:
app_train[['TARGET', 'br_variance_pop_days_enddate_fact']].corr()

In [ ]:
kde_target('br_variance_pop_days_enddate_fact', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_variance_pop_days_enddate_fact']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_variance_pop_days_enddate_fact'] = pt.transform(app_train[num_cols])
app_test['br_variance_pop_days_enddate_fact'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_variance_pop_days_enddate_fact']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_variance_pop_days_enddate_fact', app_train)

### br_min_amt_credit_max_overdue

In [ ]:
app_train[['TARGET', 'br_min_amt_credit_max_overdue']].corr()

In [ ]:
kde_target('br_min_amt_credit_max_overdue', app_train)

In [ ]:
num_cols = ['br_min_amt_credit_max_overdue']
# 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

# 変換後のデータで各列を置換
app_train['br_min_amt_credit_max_overdue'] = pt.transform(app_train[num_cols])
app_test['br_min_amt_credit_max_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
app_train[['TARGET', 'br_min_amt_credit_max_overdue']].corr()

In [ ]:
kde_target('br_min_amt_credit_max_overdue', app_train)

### br_max_amt_credit_max_overdue

In [ ]:
app_train[['TARGET', 'br_max_amt_credit_max_overdue']].corr()

In [ ]:
kde_target('br_max_amt_credit_max_overdue', app_train)

In [ ]:
num_cols = ['br_max_amt_credit_max_overdue']
# 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

# 変換後のデータで各列を置換
app_train['br_max_amt_credit_max_overdue'] = pt.transform(app_train[num_cols])
app_test['br_max_amt_credit_max_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
app_train[['TARGET', 'br_max_amt_credit_max_overdue']].corr()

In [ ]:
kde_target('br_max_amt_credit_max_overdue', app_train)

### br_sum_amt_credit_max_overdue

In [ ]:
app_train[['TARGET', 'br_sum_amt_credit_max_overdue']].corr()

In [ ]:
kde_target('br_sum_amt_credit_max_overdue', app_train)

In [ ]:
num_cols = ['br_sum_amt_credit_max_overdue']
# 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

# 変換後のデータで各列を置換
app_train['br_sum_amt_credit_max_overdue'] = pt.transform(app_train[num_cols])
app_test['br_sum_amt_credit_max_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
app_train[['TARGET', 'br_sum_amt_credit_max_overdue']].corr()

In [ ]:
kde_target('br_sum_amt_credit_max_overdue', app_train)

### br_avg_amt_credit_max_overdue

In [ ]:
app_train[['TARGET', 'br_avg_amt_credit_max_overdue']].corr()

In [ ]:
kde_target('br_avg_amt_credit_max_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_avg_amt_credit_max_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_avg_amt_credit_max_overdue'] = pt.transform(app_train[num_cols])
app_test['br_avg_amt_credit_max_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_avg_amt_credit_max_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_avg_amt_credit_max_overdue', app_train)

### br_cnt_amt_credit_max_overdue

In [ ]:
app_train[['TARGET', 'br_cnt_amt_credit_max_overdue']].corr()

In [ ]:
kde_target('br_cnt_amt_credit_max_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_cnt_amt_credit_max_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_cnt_amt_credit_max_overdue'] = pt.transform(app_train[num_cols])
app_test['br_cnt_amt_credit_max_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_cnt_amt_credit_max_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_cnt_amt_credit_max_overdue', app_train)

### br_stddev_amt_credit_max_overdue

In [ ]:
app_train[['TARGET', 'br_stddev_amt_credit_max_overdue']].corr()

In [ ]:
kde_target('br_stddev_amt_credit_max_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_amt_credit_max_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_amt_credit_max_overdue'] = pt.transform(app_train[num_cols])
app_test['br_stddev_amt_credit_max_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_amt_credit_max_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_amt_credit_max_overdue', app_train)

### br_stddev_pop_amt_credit_max_overdue

In [ ]:
app_train[['TARGET', 'br_stddev_pop_amt_credit_max_overdue']].corr()

In [ ]:
kde_target('br_stddev_pop_amt_credit_max_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_pop_amt_credit_max_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_pop_amt_credit_max_overdue'] = pt.transform(app_train[num_cols])
app_test['br_stddev_pop_amt_credit_max_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_pop_amt_credit_max_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_pop_amt_credit_max_overdue', app_train)

### br_var_pop_amt_credit_max_overdue

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_pop_amt_credit_max_overdue']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_pop_amt_credit_max_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_pop_amt_credit_max_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_pop_amt_credit_max_overdue'] = pt.transform(app_train[num_cols])
app_test['br_var_pop_amt_credit_max_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_pop_amt_credit_max_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_pop_amt_credit_max_overdue', app_train)

### br_var_samp_amt_credit_max_overdue

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_samp_amt_credit_max_overdue']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_samp_amt_credit_max_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_samp_amt_credit_max_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_samp_amt_credit_max_overdue'] = pt.transform(app_train[num_cols])
app_test['br_var_samp_amt_credit_max_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_samp_amt_credit_max_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_samp_amt_credit_max_overdue', app_train)

### br_variance_pop_amt_credit_max_overdue

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_variance_pop_amt_credit_max_overdue']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_variance_pop_amt_credit_max_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_variance_pop_amt_credit_max_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_variance_pop_amt_credit_max_overdue'] = pt.transform(app_train[num_cols])
app_test['br_variance_pop_amt_credit_max_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_variance_pop_amt_credit_max_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_variance_pop_amt_credit_max_overdue', app_train)

### br_min_cnt_credit_prolong

In [ ]:
app_train[['TARGET', 'br_min_cnt_credit_prolong']].corr()

In [ ]:
kde_target('br_min_cnt_credit_prolong', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_min_cnt_credit_prolong']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_min_cnt_credit_prolong'] = pt.transform(app_train[num_cols])
app_test['br_min_cnt_credit_prolong'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_min_cnt_credit_prolong']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_min_cnt_credit_prolong', app_train)

### br_max_cnt_credit_prolong

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_max_cnt_credit_prolong']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_max_cnt_credit_prolong', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_max_cnt_credit_prolong']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_max_cnt_credit_prolong'] = pt.transform(app_train[num_cols])
app_test['br_max_cnt_credit_prolong'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_max_cnt_credit_prolong']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_max_cnt_credit_prolong', app_train)

### br_sum_cnt_credit_prolong

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_sum_cnt_credit_prolong']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_sum_cnt_credit_prolong', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_sum_cnt_credit_prolong']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_sum_cnt_credit_prolong'] = pt.transform(app_train[num_cols])
app_test['br_sum_cnt_credit_prolong'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_sum_cnt_credit_prolong']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_sum_cnt_credit_prolong', app_train)

### br_avg_cnt_credit_prolong

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_avg_cnt_credit_prolong']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_avg_cnt_credit_prolong', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_avg_cnt_credit_prolong']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_avg_cnt_credit_prolong'] = pt.transform(app_train[num_cols])
app_test['br_avg_cnt_credit_prolong'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_avg_cnt_credit_prolong']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_avg_cnt_credit_prolong', app_train)

### br_cnt_cnt_credit_prolong

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_cnt_cnt_credit_prolong']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_cnt_cnt_credit_prolong', app_train)

### br_stddev_cnt_credit_prolong

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_stddev_cnt_credit_prolong']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_stddev_cnt_credit_prolong', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_cnt_credit_prolong']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_cnt_credit_prolong'] = pt.transform(app_train[num_cols])
app_test['br_stddev_cnt_credit_prolong'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_cnt_credit_prolong']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_cnt_credit_prolong', app_train)

### br_stddev_pop_cnt_credit_prolong

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_stddev_pop_cnt_credit_prolong']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_stddev_pop_cnt_credit_prolong', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_pop_cnt_credit_prolong']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_pop_cnt_credit_prolong'] = pt.transform(app_train[num_cols])
app_test['br_stddev_pop_cnt_credit_prolong'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_pop_cnt_credit_prolong']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_pop_cnt_credit_prolong', app_train)

### br_var_pop_cnt_credit_prolong

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_pop_cnt_credit_prolong']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_pop_cnt_credit_prolong', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_pop_cnt_credit_prolong']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_pop_cnt_credit_prolong'] = pt.transform(app_train[num_cols])
app_test['br_var_pop_cnt_credit_prolong'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_pop_cnt_credit_prolong']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_pop_cnt_credit_prolong', app_train)

### br_var_samp_cnt_credit_prolong

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_samp_cnt_credit_prolong']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_samp_cnt_credit_prolong', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_samp_cnt_credit_prolong']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_samp_cnt_credit_prolong'] = pt.transform(app_train[num_cols])
app_test['br_var_samp_cnt_credit_prolong'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_samp_cnt_credit_prolong']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_samp_cnt_credit_prolong', app_train)

### br_variance_pop_cnt_credit_prolong

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_variance_pop_cnt_credit_prolong']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_variance_pop_cnt_credit_prolong', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_variance_pop_cnt_credit_prolong']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_variance_pop_cnt_credit_prolong'] = pt.transform(app_train[num_cols])
app_test['br_variance_pop_cnt_credit_prolong'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_variance_pop_cnt_credit_prolong']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_variance_pop_cnt_credit_prolong', app_train)

### br_min_amt_credit_sum

In [ ]:
app_train[['TARGET', 'br_min_amt_credit_sum']].corr()

In [ ]:
kde_target('br_min_amt_credit_sum', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_min_amt_credit_sum']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_min_amt_credit_sum'] = pt.transform(app_train[num_cols])
app_test['br_min_amt_credit_sum'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_min_amt_credit_sum']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_min_amt_credit_sum', app_train)

### br_max_amt_credit_sum

In [ ]:
app_train[['TARGET', 'br_max_amt_credit_sum']].corr()

In [ ]:
kde_target('br_max_amt_credit_sum', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_max_amt_credit_sum']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_max_amt_credit_sum'] = pt.transform(app_train[num_cols])
app_test['br_max_amt_credit_sum'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_max_amt_credit_sum']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_max_amt_credit_sum', app_train)

### br_sum_amt_credit_sum

In [ ]:
app_train[['TARGET', 'br_sum_amt_credit_sum']].corr()

In [ ]:
kde_target('br_sum_amt_credit_sum', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_sum_amt_credit_sum']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_sum_amt_credit_sum'] = pt.transform(app_train[num_cols])
app_test['br_sum_amt_credit_sum'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_sum_amt_credit_sum']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_sum_amt_credit_sum', app_train)

### br_avg_amt_credit_sum

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_avg_amt_credit_sum']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_avg_amt_credit_sum', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_avg_amt_credit_sum']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_avg_amt_credit_sum'] = pt.transform(app_train[num_cols])
app_test['br_avg_amt_credit_sum'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_avg_amt_credit_sum']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_avg_amt_credit_sum', app_train)

### br_cnt_amt_credit_sum

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_cnt_amt_credit_sum']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_cnt_amt_credit_sum', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_cnt_amt_credit_sum']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_cnt_amt_credit_sum'] = pt.transform(app_train[num_cols])
app_test['br_cnt_amt_credit_sum'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_cnt_amt_credit_sum']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_cnt_amt_credit_sum', app_train)

### br_stddev_amt_credit_sum

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_stddev_amt_credit_sum']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_stddev_amt_credit_sum', app_train)

### br_stddev_pop_amt_credit_sum

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_stddev_pop_amt_credit_sum']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_stddev_pop_amt_credit_sum', app_train)

### br_var_pop_amt_credit_sum

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_pop_amt_credit_sum']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_pop_amt_credit_sum', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_pop_amt_credit_sum']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_pop_amt_credit_sum'] = pt.transform(app_train[num_cols])
app_test['br_var_pop_amt_credit_sum'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_pop_amt_credit_sum']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_pop_amt_credit_sum', app_train)

### br_var_samp_amt_credit_sum

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_samp_amt_credit_sum']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_samp_amt_credit_sum', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_samp_amt_credit_sum']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_samp_amt_credit_sum'] = pt.transform(app_train[num_cols])
app_test['br_var_samp_amt_credit_sum'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_samp_amt_credit_sum']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_samp_amt_credit_sum', app_train)

### br_variance_pop_amt_credit_sum

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_variance_pop_amt_credit_sum']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_variance_pop_amt_credit_sum', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_variance_pop_amt_credit_sum']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_variance_pop_amt_credit_sum'] = pt.transform(app_train[num_cols])
app_test['br_variance_pop_amt_credit_sum'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_variance_pop_amt_credit_sum']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_variance_pop_amt_credit_sum', app_train)

### br_min_amt_credit_sum_debt

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_min_amt_credit_sum_debt']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_min_amt_credit_sum_debt', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_min_amt_credit_sum_debt']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_min_amt_credit_sum_debt'] = pt.transform(app_train[num_cols])
app_test['br_min_amt_credit_sum_debt'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_min_amt_credit_sum_debt']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_min_amt_credit_sum_debt', app_train)

In [ ]:
app_train.drop(['br_min_amt_credit_sum_debt'], axis=1, inplace=True)
app_test.drop(['br_min_amt_credit_sum_debt'], axis=1, inplace=True)

### br_max_amt_credit_sum_debt

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_max_amt_credit_sum_debt']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_max_amt_credit_sum_debt', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_max_amt_credit_sum_debt']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_max_amt_credit_sum_debt'] = pt.transform(app_train[num_cols])
app_test['br_max_amt_credit_sum_debt'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_max_amt_credit_sum_debt']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_max_amt_credit_sum_debt', app_train)

### br_sum_amt_credit_sum_debt

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_sum_amt_credit_sum_debt']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_sum_amt_credit_sum_debt', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_sum_amt_credit_sum_debt']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_sum_amt_credit_sum_debt'] = pt.transform(app_train[num_cols])
app_test['br_sum_amt_credit_sum_debt'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_sum_amt_credit_sum_debt']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_sum_amt_credit_sum_debt', app_train)

In [ ]:
app_train.drop(['br_sum_amt_credit_sum_debt'], axis=1, inplace=True)
app_test.drop(['br_sum_amt_credit_sum_debt'], axis=1, inplace=True)

### br_avg_amt_credit_sum_debt

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_avg_amt_credit_sum_debt']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_avg_amt_credit_sum_debt', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_avg_amt_credit_sum_debt']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_avg_amt_credit_sum_debt'] = pt.transform(app_train[num_cols])
app_test['br_avg_amt_credit_sum_debt'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_avg_amt_credit_sum_debt']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_avg_amt_credit_sum_debt', app_train)

In [ ]:
app_train.drop(['br_avg_amt_credit_sum_debt'], axis=1, inplace=True)
app_test.drop(['br_avg_amt_credit_sum_debt'], axis=1, inplace=True)

### br_cnt_amt_credit_sum_debt

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_cnt_amt_credit_sum_debt']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_cnt_amt_credit_sum_debt', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_cnt_amt_credit_sum_debt']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_cnt_amt_credit_sum_debt'] = pt.transform(app_train[num_cols])
app_test['br_cnt_amt_credit_sum_debt'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_cnt_amt_credit_sum_debt']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_cnt_amt_credit_sum_debt', app_train)

### br_stddev_amt_credit_sum_debt

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_stddev_amt_credit_sum_debt']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_stddev_amt_credit_sum_debt', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_amt_credit_sum_debt']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_amt_credit_sum_debt'] = pt.transform(app_train[num_cols])
app_test['br_stddev_amt_credit_sum_debt'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_amt_credit_sum_debt']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_amt_credit_sum_debt', app_train)

### br_stddev_pop_amt_credit_sum_debt

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_stddev_pop_amt_credit_sum_debt']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_stddev_pop_amt_credit_sum_debt', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_sum_amt_credit_max_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_sum_amt_credit_max_overdue'] = pt.transform(app_train[num_cols])
app_test['br_sum_amt_credit_max_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_sum_amt_credit_max_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_sum_amt_credit_max_overdue', app_train)

### br_var_pop_amt_credit_sum_debt

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_pop_amt_credit_sum_debt']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_pop_amt_credit_sum_debt', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_pop_amt_credit_sum_debt']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_pop_amt_credit_sum_debt'] = pt.transform(app_train[num_cols])
app_test['br_var_pop_amt_credit_sum_debt'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_pop_amt_credit_sum_debt']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_pop_amt_credit_sum_debt', app_train)

### br_var_samp_amt_credit_sum_debt

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_samp_amt_credit_sum_debt']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_samp_amt_credit_sum_debt', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_samp_amt_credit_sum_debt']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_samp_amt_credit_sum_debt'] = pt.transform(app_train[num_cols])
app_test['br_var_samp_amt_credit_sum_debt'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_samp_amt_credit_sum_debt']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_samp_amt_credit_sum_debt', app_train)

### br_variance_pop_amt_credit_sum_debt

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_variance_pop_amt_credit_sum_debt']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_variance_pop_amt_credit_sum_debt', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_variance_pop_amt_credit_sum_debt']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_variance_pop_amt_credit_sum_debt'] = pt.transform(app_train[num_cols])
app_test['br_variance_pop_amt_credit_sum_debt'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_variance_pop_amt_credit_sum_debt']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_variance_pop_amt_credit_sum_debt', app_train)

### br_min_amt_credit_sum_limit

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_min_amt_credit_sum_limit']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_min_amt_credit_sum_limit', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_min_amt_credit_sum_limit']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_min_amt_credit_sum_limit'] = pt.transform(app_train[num_cols])
app_test['br_min_amt_credit_sum_limit'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_min_amt_credit_sum_limit']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_min_amt_credit_sum_limit', app_train)

In [ ]:
app_train.drop(['br_min_amt_credit_sum_limit'], axis=1, inplace=True)
app_test.drop(['br_min_amt_credit_sum_limit'], axis=1, inplace=True)

### br_max_amt_credit_sum_limit

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_max_amt_credit_sum_limit']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_max_amt_credit_sum_limit', app_train)

In [ ]:
## Yao-Johnson変換
#num_cols = ['br_max_amt_credit_sum_limit']
### 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
#app_train['br_max_amt_credit_sum_limit'] = pt.transform(app_train[num_cols])
#app_test['br_max_amt_credit_sum_limit'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
#app_train[['TARGET', 'br_max_amt_credit_sum_limit']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
#kde_target('br_max_amt_credit_sum_limit', app_train)

### br_sum_amt_credit_sum_limit

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_sum_amt_credit_sum_limit']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_sum_amt_credit_sum_limit', app_train)

In [ ]:
## Yao-Johnson変換
#num_cols = ['br_sum_amt_credit_sum_limit']
### 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
#app_train['br_sum_amt_credit_sum_limit'] = pt.transform(app_train[num_cols])
#app_test['br_sum_amt_credit_sum_limit'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
#app_train[['TARGET', 'br_sum_amt_credit_sum_limit']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
#kde_target('br_sum_amt_credit_sum_limit', app_train)

### br_avg_amt_credit_sum_limit

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_avg_amt_credit_sum_limit']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_avg_amt_credit_sum_limit', app_train)

In [ ]:
## Yao-Johnson変換
#num_cols = ['br_avg_amt_credit_sum_limit']
### 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
#app_train['br_avg_amt_credit_sum_limit'] = pt.transform(app_train[num_cols])
#app_test['br_avg_amt_credit_sum_limit'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
#app_train[['TARGET', 'br_avg_amt_credit_sum_limit']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
#kde_target('br_avg_amt_credit_sum_limit', app_train)

### br_cnt_amt_credit_sum_limit

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_cnt_amt_credit_sum_limit']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_cnt_amt_credit_sum_limit', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_cnt_amt_credit_sum_limit']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_cnt_amt_credit_sum_limit'] = pt.transform(app_train[num_cols])
app_test['br_cnt_amt_credit_sum_limit'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_cnt_amt_credit_sum_limit']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_cnt_amt_credit_sum_limit', app_train)

### br_stddev_amt_credit_sum_limit

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_stddev_amt_credit_sum_limit']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_stddev_amt_credit_sum_limit', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_amt_credit_sum_limit']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_amt_credit_sum_limit'] = pt.transform(app_train[num_cols])
app_test['br_stddev_amt_credit_sum_limit'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_amt_credit_sum_limit']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_amt_credit_sum_limit', app_train)

### br_stddev_pop_amt_credit_sum_limit

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_stddev_pop_amt_credit_sum_limit']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_stddev_pop_amt_credit_sum_limit', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_pop_amt_credit_sum_limit']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_pop_amt_credit_sum_limit'] = pt.transform(app_train[num_cols])
app_test['br_stddev_pop_amt_credit_sum_limit'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_pop_amt_credit_sum_limit']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_pop_amt_credit_sum_limit', app_train)

### br_var_pop_amt_credit_sum_limit

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_pop_amt_credit_sum_limit']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_pop_amt_credit_sum_limit', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_pop_amt_credit_sum_limit']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_pop_amt_credit_sum_limit'] = pt.transform(app_train[num_cols])
app_test['br_var_pop_amt_credit_sum_limit'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_pop_amt_credit_sum_limit']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_pop_amt_credit_sum_limit', app_train)

### br_var_samp_amt_credit_sum_limit

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_samp_amt_credit_sum_limit']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_samp_amt_credit_sum_limit', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_samp_amt_credit_sum_limit']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_samp_amt_credit_sum_limit'] = pt.transform(app_train[num_cols])
app_test['br_var_samp_amt_credit_sum_limit'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_samp_amt_credit_sum_limit']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_samp_amt_credit_sum_limit', app_train)

### br_variance_pop_amt_credit_sum_limit

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_variance_pop_amt_credit_sum_limit']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_variance_pop_amt_credit_sum_limit', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_variance_pop_amt_credit_sum_limit']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_variance_pop_amt_credit_sum_limit'] = pt.transform(app_train[num_cols])
app_test['br_variance_pop_amt_credit_sum_limit'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_variance_pop_amt_credit_sum_limit']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_variance_pop_amt_credit_sum_limit', app_train)

### br_min_amt_credit_sum_overdue

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_min_amt_credit_sum_overdue']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_min_amt_credit_sum_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_min_amt_credit_sum_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_min_amt_credit_sum_overdue'] = pt.transform(app_train[num_cols])
app_test['br_min_amt_credit_sum_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_min_amt_credit_sum_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_min_amt_credit_sum_overdue', app_train)

In [ ]:
app_train.drop(['br_min_amt_credit_sum_overdue'], axis=1, inplace=True)
app_test.drop(['br_min_amt_credit_sum_overdue'], axis=1, inplace=True)

### br_max_amt_credit_sum_overdue

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_max_amt_credit_sum_overdue']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_max_amt_credit_sum_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_max_amt_credit_sum_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_max_amt_credit_sum_overdue'] = pt.transform(app_train[num_cols])
app_test['br_max_amt_credit_sum_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_max_amt_credit_sum_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_max_amt_credit_sum_overdue', app_train)

### br_sum_amt_credit_sum_overdue

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_sum_amt_credit_sum_overdue']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_sum_amt_credit_sum_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_sum_amt_credit_sum_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_sum_amt_credit_sum_overdue'] = pt.transform(app_train[num_cols])
app_test['br_sum_amt_credit_sum_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_sum_amt_credit_sum_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_sum_amt_credit_sum_overdue', app_train)

### br_avg_amt_credit_sum_overdue

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_avg_amt_credit_sum_overdue']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_avg_amt_credit_sum_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_avg_amt_credit_sum_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_avg_amt_credit_sum_overdue'] = pt.transform(app_train[num_cols])
app_test['br_avg_amt_credit_sum_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_avg_amt_credit_sum_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_avg_amt_credit_sum_overdue', app_train)

### br_cnt_amt_credit_sum_overdue

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_cnt_amt_credit_sum_overdue']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_cnt_amt_credit_sum_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_cnt_amt_credit_sum_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_cnt_amt_credit_sum_overdue'] = pt.transform(app_train[num_cols])
app_test['br_cnt_amt_credit_sum_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_cnt_amt_credit_sum_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_cnt_amt_credit_sum_overdue', app_train)

### br_stddev_amt_credit_sum_overdue

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_stddev_amt_credit_sum_overdue']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_stddev_amt_credit_sum_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_amt_credit_sum_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_amt_credit_sum_overdue'] = pt.transform(app_train[num_cols])
app_test['br_stddev_amt_credit_sum_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_amt_credit_sum_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_amt_credit_sum_overdue', app_train)

### br_stddev_pop_amt_credit_sum_overdue

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_stddev_pop_amt_credit_sum_overdue']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_stddev_pop_amt_credit_sum_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_pop_amt_credit_sum_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_pop_amt_credit_sum_overdue'] = pt.transform(app_train[num_cols])
app_test['br_stddev_pop_amt_credit_sum_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_pop_amt_credit_sum_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_pop_amt_credit_sum_overdue', app_train)

### br_var_pop_amt_credit_sum_overdue

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_pop_amt_credit_sum_overdue']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_pop_amt_credit_sum_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_pop_amt_credit_sum_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_pop_amt_credit_sum_overdue'] = pt.transform(app_train[num_cols])
app_test['br_var_pop_amt_credit_sum_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_pop_amt_credit_sum_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_pop_amt_credit_sum_overdue', app_train)

### br_var_samp_amt_credit_sum_overdue

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_samp_amt_credit_sum_overdue']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_samp_amt_credit_sum_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_samp_amt_credit_sum_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_samp_amt_credit_sum_overdue'] = pt.transform(app_train[num_cols])
app_test['br_var_samp_amt_credit_sum_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_samp_amt_credit_sum_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_samp_amt_credit_sum_overdue', app_train)

### br_variance_pop_amt_credit_sum_overdue

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_variance_pop_amt_credit_sum_overdue']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_variance_pop_amt_credit_sum_overdue', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_variance_pop_amt_credit_sum_overdue']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_variance_pop_amt_credit_sum_overdue'] = pt.transform(app_train[num_cols])
app_test['br_variance_pop_amt_credit_sum_overdue'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_variance_pop_amt_credit_sum_overdue']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_variance_pop_amt_credit_sum_overdue', app_train)

### br_min_days_credit_update

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_min_days_credit_update']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_min_days_credit_update', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_min_days_credit_update']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_min_days_credit_update'] = pt.transform(app_train[num_cols])
app_test['br_min_days_credit_update'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_min_days_credit_update']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_min_days_credit_update', app_train)

### br_max_days_credit_update

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_max_days_credit_update']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_max_days_credit_update', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_max_days_credit_update']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_max_days_credit_update'] = pt.transform(app_train[num_cols])
app_test['br_max_days_credit_update'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_max_days_credit_update']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_max_days_credit_update', app_train)

### br_sum_days_credit_update

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_sum_days_credit_update']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_sum_days_credit_update', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_sum_days_credit_update']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_sum_days_credit_update'] = pt.transform(app_train[num_cols])
app_test['br_sum_days_credit_update'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_sum_days_credit_update']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_sum_days_credit_update', app_train)

### br_avg_days_credit_update

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_avg_days_credit_update']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_avg_days_credit_update', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_avg_days_credit_update']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_avg_days_credit_update'] = pt.transform(app_train[num_cols])
app_test['br_avg_days_credit_update'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_avg_days_credit_update']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_avg_days_credit_update', app_train)

### br_cnt_days_credit_update

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_cnt_days_credit_update']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_cnt_days_credit_update', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_cnt_days_credit_update']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_cnt_days_credit_update'] = pt.transform(app_train[num_cols])
app_test['br_cnt_days_credit_update'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_cnt_days_credit_update']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_cnt_days_credit_update', app_train)

### br_stddev_days_credit_update

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_stddev_days_credit_update']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_stddev_days_credit_update', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_days_credit_update']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_days_credit_update'] = pt.transform(app_train[num_cols])
app_test['br_stddev_days_credit_update'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_days_credit_update']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_days_credit_update', app_train)

### br_stddev_pop_days_credit_update

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_stddev_pop_days_credit_update']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_stddev_pop_days_credit_update', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_pop_days_credit_update']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_pop_days_credit_update'] = pt.transform(app_train[num_cols])
app_test['br_stddev_pop_days_credit_update'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_pop_days_credit_update']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_pop_days_credit_update', app_train)

### br_var_pop_days_credit_update

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_pop_days_credit_update']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_pop_days_credit_update', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_pop_days_credit_update']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_pop_days_credit_update'] = pt.transform(app_train[num_cols])
app_test['br_var_pop_days_credit_update'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_pop_days_credit_update']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_pop_days_credit_update', app_train)

### br_var_samp_days_credit_update

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_samp_days_credit_update']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_samp_days_credit_update', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_samp_days_credit_update']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_samp_days_credit_update'] = pt.transform(app_train[num_cols])
app_test['br_var_samp_days_credit_update'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_samp_days_credit_update']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_samp_days_credit_update', app_train)

### br_variance_pop_days_credit_update

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_variance_pop_days_credit_update']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_variance_pop_days_credit_update', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_variance_pop_days_credit_update']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_variance_pop_days_credit_update'] = pt.transform(app_train[num_cols])
app_test['br_variance_pop_days_credit_update'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_variance_pop_days_credit_update']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_variance_pop_days_credit_update', app_train)

### br_min_amt_annuity

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_min_amt_annuity']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_min_amt_annuity', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_min_amt_annuity']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_min_amt_annuity'] = pt.transform(app_train[num_cols])
app_test['br_min_amt_annuity'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_min_amt_annuity']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_min_amt_annuity', app_train)

### br_max_amt_annuity

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_max_amt_annuity']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_max_amt_annuity', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_max_amt_annuity']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_max_amt_annuity'] = pt.transform(app_train[num_cols])
app_test['br_max_amt_annuity'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_max_amt_annuity']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_max_amt_annuity', app_train)

### br_sum_amt_annuity

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_sum_amt_annuity']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_sum_amt_annuity', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_sum_amt_annuity']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_sum_amt_annuity'] = pt.transform(app_train[num_cols])
app_test['br_sum_amt_annuity'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_sum_amt_annuity']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_sum_amt_annuity', app_train)

### br_avg_amt_annuity

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_avg_amt_annuity']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_avg_amt_annuity', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_avg_amt_annuity']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_avg_amt_annuity'] = pt.transform(app_train[num_cols])
app_test['br_avg_amt_annuity'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_avg_amt_annuity']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_avg_amt_annuity', app_train)

### br_cnt_amt_annuity

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_cnt_amt_annuity']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_cnt_amt_annuity', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_cnt_amt_annuity']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_cnt_amt_annuity'] = pt.transform(app_train[num_cols])
app_test['br_cnt_amt_annuity'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_cnt_amt_annuity']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_cnt_amt_annuity', app_train)

### br_stddev_amt_annuity

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_stddev_amt_annuity']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_stddev_amt_annuity', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_amt_annuity']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_amt_annuity'] = pt.transform(app_train[num_cols])
app_test['br_stddev_amt_annuity'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_amt_annuity']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_amt_annuity', app_train)

### br_stddev_pop_amt_annuity

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_stddev_pop_amt_annuity']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_stddev_pop_amt_annuity', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_stddev_pop_amt_annuity']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_stddev_pop_amt_annuity'] = pt.transform(app_train[num_cols])
app_test['br_stddev_pop_amt_annuity'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_stddev_pop_amt_annuity']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_stddev_pop_amt_annuity', app_train)

### br_var_pop_amt_annuity

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_pop_amt_annuity']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_pop_amt_annuity', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_pop_amt_annuity']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_pop_amt_annuity'] = pt.transform(app_train[num_cols])
app_test['br_var_pop_amt_annuity'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_pop_amt_annuity']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_pop_amt_annuity', app_train)

### br_var_samp_amt_annuity

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_var_samp_amt_annuity']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_var_samp_amt_annuity', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_var_samp_amt_annuity']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_var_samp_amt_annuity'] = pt.transform(app_train[num_cols])
app_test['br_var_samp_amt_annuity'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_var_samp_amt_annuity']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_var_samp_amt_annuity', app_train)

### br_variance_pop_amt_annuity

In [ ]:
## 目的変数との相関
app_train[['TARGET', 'br_variance_pop_amt_annuity']].corr()

In [ ]:
## 特徴量の分布
kde_target('br_variance_pop_amt_annuity', app_train)

In [ ]:
## Yao-Johnson変換
num_cols = ['br_variance_pop_amt_annuity']
### 学習データに基づいてYao-Johnson変換を定義
pt = PowerTransformer(method = 'yeo-johnson')
pt.fit(app_train[num_cols].copy())

### 変換後のデータで各列を置換
app_train['br_variance_pop_amt_annuity'] = pt.transform(app_train[num_cols])
app_test['br_variance_pop_amt_annuity'] = pt.transform(app_test[num_cols])

In [ ]:
## Yao-Johnson変換後の目的変数との相関
app_train[['TARGET', 'br_variance_pop_amt_annuity']].corr()

In [ ]:
## Yao-Johnson変換後の特徴量の分布
kde_target('br_variance_pop_amt_annuity', app_train)

## CSVをEXPORT

In [ ]:
### train
app_train.to_csv(
    path_or_buf="./home-credit-default-risk/exports/hc_br_train.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [ ]:
### test
app_test.to_csv(
    path_or_buf="./home-credit-default-risk/exports/hc_br_test.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [ ]:
columns = list(app_train.columns)
columns.remove('SK_ID_CURR')
columns.remove('TARGET')

In [ ]:
data = pd.concat([app_train, app_test], sort=False)
train = data[:len(app_train)]
test = data[len(app_test):]


y_train = train['TARGET']
X_train = app_train.copy().drop('TARGET', axis=1)
X_test = app_test.copy()

In [ ]:
import xgboost as xgb
from sklearn.metrics import log_loss

y_preds = []
models = []
scores = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = columns

params = {
    'booster': 'gbtree'
    ,'objective': 'binary:logistic'
    ,'silent': 1
    ,'random_state': 17
    ,'gpu_id': 0
    ,'tree_method': 'gpu_hist'
    ,'eval_metric': 'auc'
    ,'eta': 0.1
    #,'max_round': 100
    ,'max_depth': 8
    ,'subsample': 0.8
    ,'colsample_state': 71
    ,'colsample_bytree': 1.0
    ,'colsample_bylevel': 0.3
    ,'subsumple': 0.9
    ,'gamma': 0
    ,'lambda': 1
    ,'alpha': 0
    ,'min_child_weight': 1
}

num_round = 50

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)
    
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round, evals=watchlist)
    
    va_pred = model.predict(dvalid)
    score = log_loss(y_val, va_pred)
    oof_train[valid_index] = va_pred
    
    y_pred = model.predict(dtest) 
    
    
    #oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    #y_pred = model.predict(X_test, num_iteration=model.best_iteration)##

    y_preds.append(y_pred)
    models.append(model)
    scores.append(score)

In [ ]:
app_train = app_train.drop('TARGET', axis=1)

In [ ]:
# trainへマージ
app_train = pd.merge(app_train_origin.copy(), app_br_train, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test_origin.copy(), app_br_test, on='SK_ID_CURR', how='left')
len(app_test)

In [ ]:
columns = list(app_train.columns)
columns.remove('SK_ID_CURR')
columns.remove('TARGET')

In [ ]:
data = pd.concat([app_train, app_test], sort=False)
train = data[:len(app_train)]
test = data[len(app_test):]

y_train = train['TARGET']
X_train = app_train.copy().drop('TARGET', axis=1)
X_test = app_test.copy()

In [ ]:
import xgboost as xgb
from sklearn.metrics import log_loss

y_preds = []
models = []
scores = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = columns

params = {
    'booster': 'gbtree'
    ,'objective': 'binary:logistic'
    ,'silent': 1
    ,'random_state': 17
    ,'gpu_id': 0
    ,'tree_method': 'gpu_hist'
    ,'eval_metric': 'auc'
    ,'eta': 0.1
    #,'max_round': 100
    ,'max_depth': 8
    ,'subsample': 0.8
    ,'colsample_state': 71
    ,'colsample_bytree': 1.0
    ,'colsample_bylevel': 0.3
    ,'subsumple': 0.9
    ,'gamma': 0
    ,'lambda': 1
    ,'alpha': 0
    ,'min_child_weight': 1
}

num_round = 50

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)
    
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round, evals=watchlist)
    
    va_pred = model.predict(dvalid)
    score = log_loss(y_val, va_pred)
    oof_train[valid_index] = va_pred
    
    y_pred = model.predict(dtest) 
    
    
    #oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    #y_pred = model.predict(X_test, num_iteration=model.best_iteration)##

    y_preds.append(y_pred)
    models.append(model)
    scores.append(score)

In [ ]:
# CSV-00
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[0]
submit_all.isnull().sum()

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_00.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [ ]:
# CSV-01
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[1]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_01.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [ ]:
# CSV-02
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[2]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_02.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [ ]:
# CSV-03
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[3]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_03.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [ ]:
# CSV-04
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[4]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_04.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [ ]:
submit_all_00 = pd.read_csv('./home-credit-default-risk/submit/submit_all_00.csv')
submit_all_01 = pd.read_csv('./home-credit-default-risk/submit/submit_all_01.csv')
submit_all_02 = pd.read_csv('./home-credit-default-risk/submit/submit_all_02.csv')
submit_all_03 = pd.read_csv('./home-credit-default-risk/submit/submit_all_03.csv')
submit_all_04 = pd.read_csv('./home-credit-default-risk/submit/submit_all_04.csv')

In [ ]:
submit_all = pd.DataFrame({
    'submit_all_00': submit_all_00['TARGET'].values
    ,'submit_all_01': submit_all_01['TARGET'].values
    ,'submit_all_02': submit_all_02['TARGET'].values
    ,'submit_all_03': submit_all_03['TARGET'].values
    ,'submit_all_04': submit_all_04['TARGET'].values
    })
submit_all.head()

In [ ]:
submit_all = app_test[["SK_ID_CURR"]].copy()
submit_all['TARGET'] = (submit_all_00['TARGET'] + submit_all_01['TARGET'] + submit_all_02['TARGET'] + submit_all_03['TARGET'] + submit_all_04['TARGET'])/5
submit_all.head()

In [ ]:
submit_all['TARGET'] = (submit_all['TARGET'] / 5)
submit_all.to_csv('./home-credit-default-risk/submit/submit_br_main_test_12-monthes.csv', index=False)
submit_all.head()

In [ ]:
submit_all.count()